In [37]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D
import matplotlib

# 3D plot
matplotlib.use('TkAgg')

In [38]:
data_path = '/Users/aboba/Desktop/isp/B_r.dat'
data_0 = np.loadtxt(data_path, usecols=(0, 1, 2))

# Используем список для временного хранения данных
temp_data = []

for i in range(len(data_0)):
    if  (data_0[i][0] > 0 ):
        temp_data.append(data_0[i, :])

# Преобразуем список обратно в двумерный массив NumPy
data = np.array(temp_data)
len(data_0)



72

In [39]:
# Теперь вы можете без проблем извлечь x_data, y_data, z_data
x_data, y_data, z_data = data[:, 0], data[:, 1], data[:, 2]

In [40]:
def f0(x, y): return 1
def f1(x, y): return 2
def f2(x, y): return -0.5 * y**2 + x**2
def f3(x, y): return -1.5 * x * y**2 + x**3
def f4(x, y): return 3/8 * y**4 - 3 * y**2 * x**2 + x**4
def f5(x, y): return 15/8 * x * y**4 - 5 * x**3 * y**2 + x**5
def f6(x, y): return -5/16 * y**6 + 45/8 * y**4 * x**2 - 15/2 * y**2 * x**4 + x**6
def f8(x, y): return 2688/983 * y**8 - 35/4 * y**6 * x**2 + 105/4 * y**4 * x**4 - 14 * y**2 * x**6 + x**8

In [41]:

# Функция создания модели
def create_F2_model(functions):
    def model(xy, *coefficients):
        x, y = xy
        result = 0
        for j in range(len(functions)):
            f= functions[j]
            a=coefficients[j+1]
            x_0=coefficients[0]
            result += a * f(x-x_0, y)
        return result
    return model

In [42]:
# Выбор функций и начальных предположений для коэффициентов
functions = [f0,f2,f4,f6,f8] #,f2,f4,f6
initial_guess = [4.321709729499605,0.4665555022889368,0.0005024081505509558,0,0,0]  # 0, A0, A2, A3 ,0,0,0


In [43]:
# Создание модели F2
F2_model = create_F2_model(functions)

# Выполнение подгонки
popt, pcov = curve_fit(F2_model, (x_data, y_data), z_data, p0=initial_guess, maxfev=50000)

# Результаты подгонки и их стандартные отклонения
coefficients_fit = popt
errors = np.sqrt(np.diag(pcov))

# Форматированный вывод результатов подгонки и их отклонений в процентах
for i, (coef, error) in enumerate(zip(coefficients_fit, errors)):
      if i<1:
        percent_error = (error / coef * 100) if coef != 0 else float('inf')
        print(f"A{i} = {coef} ± {error} ({percent_error:.2f}%)")
      else:   
        percent_error = (error / coef * 100) if coef != 0 else float('inf')
        
        print(f"A{i+1} = {coef} ± {error} ({percent_error:.2f}%)")

A0 = 7.338572615070551 ± 0.13628013962215377 (1.86%)
A2 = 0.00792223057350305 ± 0.0011809145321450315 (14.91%)
A3 = 0.0010085475744786964 ± 5.0542612866972224e-05 (5.01%)
A4 = -1.83391819693839e-05 ± 1.168675358365232e-06 (-6.37%)
A5 = 1.7970470333249989e-07 ± 3.0448019260154085e-08 (16.94%)
A6 = -7.590209688369808e-10 ± 1.52259033336645e-10 (-20.06%)


In [44]:
mass=np.array([0.0,0.0,0.0,0.0,0.0,0.0])
x=1.15
y=0.0
for i, (coef, error) in enumerate(zip(coefficients_fit, errors)):
        percent_error = (error / coef * 100) if coef != 0 else float('inf')
        mass[i]=coef
for i in range(len(mass)):
        if i==1:
                mass[i]=mass[i]*f0(x,y)
        if i==2:
                mass[i]=mass[i]*f2(x,y)
        if i==3:
                mass[i]=mass[i]*f4(x,y)     
        if i==4:
                mass[i]=mass[i]*f6(x,y)
print(mass)   
np.sum(mass)-mass[0]

[ 7.33857262e+00  7.92223057e-03  1.33380417e-03 -3.20753439e-05
  4.15667899e-07 -7.59020969e-10]


0.009224374305744298

In [45]:
# Визуализация результатов
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Точки данных
ax.scatter(x_data, y_data, z_data, color='b', label='Исходные данные')

# Подогнанная поверхность
x_range = np.linspace(min(x_data), max(x_data), 30)
y_range = np.linspace(min(y_data), max(y_data), 30)
x_grid, y_grid = np.meshgrid(x_range, y_range)
z_fit = F2_model((x_grid, y_grid), *coefficients_fit)

ax.plot_surface(x_grid, y_grid, z_fit, color='r', alpha=0.5, label='Подогнанная модель')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')


plt.show()


: 